In [1]:
import sys, os
sys.path.append(os.path.abspath('../src'))
from fliqe import FLIQE
import cv2
from torchvision import transforms
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(f"Using device: {device}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

img_path = "../data/video_frames_4/frame_0000.jpg"
img = cv2.imread(img_path) 


2.8.0+cu129
True
12.9
91002
Using device: cuda


In [2]:
from pyiqa.archs.niqe_arch import NIQE
from pyiqa.archs.piqe_arch import PIQE
from pyiqa.archs.brisque_arch import BRISQUE
from pyiqa.archs.arniqa_arch import ARNIQA
from pyiqa.archs.paq2piq_arch import PAQ2PIQ
from pyiqa.archs.musiq_arch import MUSIQ
from pyiqa.archs.dbcnn_arch import DBCNN
from pyiqa.archs.clipiqa_arch import CLIPIQA
from pyiqa.archs.maniqa_arch import MANIQA
from torchvision import transforms

img_tensor = transforms.ToTensor()(img).unsqueeze(0)

niqe = NIQE()
niqe_score = niqe(img_tensor)
print("NIQE score (lower better):", float(niqe_score))

piqe = PIQE()
piqe_score = piqe(img_tensor)
print("PIQE score (lower better):", float(piqe_score))

brisque = BRISQUE()
brisque_score = brisque(img_tensor)
print("BRISQUE score (lower better):", float(brisque_score.detach()))

arniqa = ARNIQA()
img_tensor = img_tensor.to(device)
arniqa = arniqa.to(device)
arniqa_score = arniqa(img_tensor)
print("ARNIQA score (higher better):", float(arniqa_score.detach()))

paq2piq = PAQ2PIQ()
paq2piq.eval()  # Ensure model is in evaluation mode
img_tensor = img_tensor.to(device)
paq2piq = paq2piq.to(device)
paq2piq_score = paq2piq(img_tensor)
print("PAQ2PIQ score (higher better):", float(paq2piq_score.detach()))

musiq = MUSIQ()
musiq.eval()  # Ensure model is in evaluation mode
img_tensor = img_tensor.to(device)
musiq = musiq.to(device)
musiq_score = musiq(img_tensor)
print("MUSIQ score (higher better):", float(musiq_score.detach()))

dbcnn = DBCNN()
dbcnn.eval()  # Ensure model is in evaluation mode
img_tensor = img_tensor.to(device)
dbcnn = dbcnn.to(device)
dbcnn_score = dbcnn(img_tensor)
print("DBCNN score (higher better):", float(dbcnn_score.detach()))

clipiqa = CLIPIQA()
clipiqa.eval()  # Ensure model is in evaluation mode
img_tensor = img_tensor.to(device)
clipiqa = clipiqa.to(device)
clipiqa_score = clipiqa(img_tensor)
print("CLIPIQA score (higher better):", float(clipiqa_score.detach()))

# maniqa = MANIQA()
# maniqa.eval()  # Ensure model is in evaluation mode
# img_tensor = img_tensor.to(device)
# maniqa = maniqa.to(device)
# maniqa_score = maniqa(img_tensor)
# print("MANIQA score (higher better):", float(maniqa_score.detach()))

fliqe = FLIQE(quality_model_path="../models/encoder_with_binary_head.pth")
FLIQE_score = fliqe.estimate_image_quality(img)
print("FLIQE score (higher better):", FLIQE_score)

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)), # 224 or 384
    transforms.ToTensor(),
])
img_tensor = transform(img).unsqueeze(0)
img_tensor = img_tensor.to(device)
FLIQE_score_tensor = fliqe.quality_model(img_tensor).item()
print("FLIQE score from tensor (higher better):", FLIQE_score_tensor)



c:\Users\k_chi\vscodeProjects\FLIr Image Quality Estimator (FLIQE)\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\k_chi\vscodeProjects\FLIr Image Quality Estimator (FLIQE)\.venv\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


NIQE score (lower better): 3.859058813628418
PIQE score (lower better): 29.576688766479492
BRISQUE score (lower better): 10.10565185546875
ARNIQA score (higher better): 0.5651703476905823
Loading pretrained model PAQ2PIQ from C:\Users\k_chi\.cache\torch\hub\pyiqa\P2P_RoIPoolModel-fit.10.bs.120-ca69882e.pth
PAQ2PIQ score (higher better): 68.78011322021484
Loading pretrained model MUSIQ from C:\Users\k_chi\.cache\torch\hub\pyiqa\musiq_ava_ckpt-e8d3f067.pth
MUSIQ score (higher better): 4.8943681716918945
Loading pretrained model SCNN from C:\Users\k_chi\.cache\torch\hub\pyiqa\DBCNN_scnn-7ea73d75.pth
Loading pretrained model DBCNN from C:\Users\k_chi\.cache\torch\hub\pyiqa\DBCNN_KonIQ10k-2de81c0a.pth
DBCNN score (higher better): 0.44894492626190186
CLIPIQA score (higher better): 0.38447484374046326
FLIQE score (higher better): 0.9513196349143982
FLIQE score from tensor (higher better): -2.9725747108459473


In [3]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import torch.nn as nn
from tqdm import tqdm
import sys, os
sys.path.append(os.path.abspath('../src'))
from fliqe.datasets import ImageDataset
from fliqe.distortions import *
from torchvision import transforms
from torch.utils.data import DataLoader

    
distortions = [Clean(), LensBlur(), MotionBlur(), GaussianNoise(), Overexposure(), Underexposure(), Compression(), Ghosting(), Aliasing()]
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)), # 224 or 384
    transforms.ToTensor(),
])

# # All folders with images
# # all_folders = [f"../data/video_frames_{i}" for i in range(1, 13)]
# # all_folders = ["../data/FLIR_ADAS_v2/video_thermal_test/data"]
# all_folders = ["../data/FLIR_ADAS_v2/images_thermal_train/data"]
# # Gather all image paths
# all_image_paths = []
# for folder in all_folders:
#     if not os.path.isdir(folder):
#         print(f"Warning: folder {folder} not found, skipping.")
#         continue
#     all_image_paths.extend(
#         [os.path.join(folder, fname)
#          for fname in os.listdir(folder)
#          if fname.lower().endswith(('.jpg', '.png'))]
#     )
# print(f"Total images found: {len(all_image_paths)}")
# # Keep randomly 10000 images
# # all_image_paths = random.sample(all_image_paths, min(10000, len(all_image_paths)))
# # print(f"Total random images: {len(all_image_paths)}")
# # Shuffle and split
# # random.shuffle(all_image_paths)
# split_ratio = 0.8  # 80% train, 20% eval
# split_idx = int(len(all_image_paths) * split_ratio)
# train_paths = all_image_paths[:split_idx]
# eval_paths = all_image_paths[split_idx:]
# # Create datasets
# train_dataset = ImageDataset(train_paths, distortions=distortions, transform=transform, binary_labels=True)
# eval_dataset = ImageDataset(eval_paths, distortions=distortions, transform=transform, binary_labels=True)
# # Create dataloaders
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
# eval_dataloader = DataLoader(eval_dataset, batch_size=32, shuffle=False, num_workers=4)
# print(f"Train Dataset length: {len(train_dataset)}")
# print(f"Eval Dataset length: {len(eval_dataset)}")

image_folder = "../data/FLIR_ADAS_v2/images_thermal_train/data"
image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder) if fname.endswith(('.jpg', '.png'))]
train_dataset = ImageDataset(image_paths, distortions=distortions, transform=transform, binary_labels=True)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
print(f"Train Dataset length: {len(train_dataset)}")
# image_folder = "../data/FLIR_ADAS_v2/video_thermal_test/data"
image_folder = "../data/FLIR_ADAS_v2/images_thermal_val/data"
image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder) if fname.endswith(('.jpg', '.png'))]
eval_dataset = ImageDataset(image_paths, distortions=distortions, transform=transform, binary_labels=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=32, shuffle=False, num_workers=4)
print(f"Eval Dataset length: {len(eval_dataset)}")

Train Dataset length: 10742
Eval Dataset length: 1144


In [4]:
# Initialize all comparison algorithms
comparison_algos = {
    "FLIQE": {"train_scores": [], "train_labels": [], "eval_scores": [], "eval_labels": [], "model": fliqe.quality_model},
    "NIQE": {"train_scores": [], "train_labels": [], "eval_scores": [], "eval_labels": [], "model": NIQE()},
    "PIQE": {"train_scores": [], "train_labels": [], "eval_scores": [], "eval_labels": [], "model": PIQE()},
    "BRISQUE": {"train_scores": [], "train_labels": [], "eval_scores": [], "eval_labels": [], "model": BRISQUE()},
    "ARNIQA": {"train_scores": [], "train_labels": [], "eval_scores": [], "eval_labels": [], "model": ARNIQA().to(device)},
    "PAQ2PIQ": {"train_scores": [], "train_labels": [], "eval_scores": [], "eval_labels": [], "model": PAQ2PIQ().eval().to(device)},
    "MUSIQ": {"train_scores": [], "train_labels": [], "eval_scores": [], "eval_labels": [], "model": MUSIQ().eval().to(device)},
    "DBCNN": {"train_scores": [], "train_labels": [], "eval_scores": [], "eval_labels": [], "model": DBCNN().eval().to(device)},
    "CLIPIQA": {"train_scores": [], "train_labels": [], "eval_scores": [], "eval_labels": [], "model": CLIPIQA().eval().to(device)}
}

def compute_scores_for_all_algos(dataloader, algo_dict, data_type="train"):
    for imgs, labels in tqdm(dataloader, desc=f"Computing {data_type} Scores", leave=False):
        imgs = imgs.to(device)
        binary_labels = torch.tensor(
            [0.0 if l == 'Clean' else 1.0 for l in labels],
            dtype=torch.float32, device=device
        ).unsqueeze(1)
        
        for algo_name, algo_data in algo_dict.items():
            model = algo_data["model"]
            scores = model(imgs)
            
            # Store scores and labels for each algorithm
            algo_data[f"{data_type}_scores"].extend(scores.detach().cpu().numpy().flatten())
            algo_data[f"{data_type}_labels"].extend(binary_labels.cpu().numpy().flatten())

# Compute scores for all algorithms on both datasets
compute_scores_for_all_algos(train_dataloader, comparison_algos, "train")
compute_scores_for_all_algos(eval_dataloader, comparison_algos, "eval")

# Print dataset sizes for verification
for algo_name in comparison_algos.keys():
    train_len = len(comparison_algos[algo_name]["train_scores"])
    eval_len = len(comparison_algos[algo_name]["eval_scores"])
    print(f"{algo_name} - Train: {train_len} scores, Eval: {eval_len} scores")

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print("\n" + "="*80)
print("ALGORITHM COMPARISON RESULTS")
print("="*80)

# Evaluate each algorithm
results_summary = {}

for algo_name, algo_data in comparison_algos.items():
    print(f"\n{algo_name} Results:")
    print("-" * 50)
    
    # Convert lists to numpy arrays
    train_scores = np.array(algo_data["train_scores"]).reshape(-1, 1)
    train_labels = np.array(algo_data["train_labels"])
    
    eval_scores = np.array(algo_data["eval_scores"]).reshape(-1, 1)
    eval_labels = np.array(algo_data["eval_labels"])
    
    # Step 1: Fit logistic regression for Platt scaling
    platt_model = LogisticRegression()
    platt_model.fit(train_scores, train_labels)
    
    # Step 2: Apply scaling to get calibrated probabilities
    calibrated_probs = platt_model.predict_proba(eval_scores)[:, 1]
    
    threshold = 0.5
    eval_preds = (calibrated_probs >= threshold).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(eval_labels, eval_preds)
    precision = precision_score(eval_labels, eval_preds)
    recall = recall_score(eval_labels, eval_preds)
    f1 = f1_score(eval_labels, eval_preds)
    
    # Store results for comparison
    results_summary[algo_name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
    
    print(f"Accuracy : {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall   : {recall:.3f}")
    print(f"F1-score : {f1:.3f}")
    
    print("\nDetailed Classification Report:")
    print(classification_report(eval_labels, eval_preds, digits=3))

# Summary comparison table
print("\n" + "="*80)
print("SUMMARY COMPARISON TABLE")
print("="*80)
print(f"{'Algorithm':<12} {'Accuracy':<10} {'Precision':<12} {'Recall':<10} {'F1-Score':<10}")
print("-" * 60)

for algo_name, metrics in results_summary.items():
    print(f"{algo_name:<12} {metrics['accuracy']:<10.3f} {metrics['precision']:<12.3f} {metrics['recall']:<10.3f} {metrics['f1']:<10.3f}")

# Find best performing algorithm
best_f1_algo = max(results_summary.keys(), key=lambda x: results_summary[x]['f1'])
print(f"\nBest performing algorithm (by F1-score): {best_f1_algo} (F1: {results_summary[best_f1_algo]['f1']:.3f})")

Loading pretrained model PAQ2PIQ from C:\Users\k_chi\.cache\torch\hub\pyiqa\P2P_RoIPoolModel-fit.10.bs.120-ca69882e.pth
Loading pretrained model MUSIQ from C:\Users\k_chi\.cache\torch\hub\pyiqa\musiq_ava_ckpt-e8d3f067.pth
Loading pretrained model SCNN from C:\Users\k_chi\.cache\torch\hub\pyiqa\DBCNN_scnn-7ea73d75.pth
Loading pretrained model DBCNN from C:\Users\k_chi\.cache\torch\hub\pyiqa\DBCNN_KonIQ10k-2de81c0a.pth


FLIQE - Train: 10742 scores, Eval: 1144 scores
NIQE - Train: 10742 scores, Eval: 1144 scores
PIQE - Train: 10742 scores, Eval: 1144 scores
BRISQUE - Train: 10742 scores, Eval: 1144 scores
ARNIQA - Train: 10742 scores, Eval: 1144 scores
PAQ2PIQ - Train: 10742 scores, Eval: 1144 scores
MUSIQ - Train: 10742 scores, Eval: 1144 scores
DBCNN - Train: 10742 scores, Eval: 1144 scores
CLIPIQA - Train: 10742 scores, Eval: 1144 scores

ALGORITHM COMPARISON RESULTS

FLIQE Results:
--------------------------------------------------
Accuracy : 0.859
Precision: 0.898
Recall   : 0.773
F1-score : 0.831

Detailed Classification Report:
              precision    recall  f1-score   support

         0.0      0.835     0.929     0.879       632
         1.0      0.898     0.773     0.831       512

    accuracy                          0.859      1144
   macro avg      0.866     0.851     0.855      1144
weighted avg      0.863     0.859     0.858      1144


NIQE Results:
--------------------------------